In [189]:
#Importamos la librería
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
import tqdm
import time

In [190]:
# Definimos el url de la pagina a consultar
url = "https://www.vega.pe/nuestras-tiendas"
# Obtenemos el html de la página que queremos consultar
response =  requests.get(url)
# Verificamos si la consulta es correcta, status_code= 200
contenido_pagina = ""
if response.status_code == 200:
    contenido_pagina = response.content
print(contenido_pagina)

b'\n<!DOCTYPE html>\n  <html lang="es-PE">\n  <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="generator" content="vtex.render-server@8.168.0">\n    <meta charset=\'utf-8\'>\n    \n  <script>\n    document.cookie = "VtexWorkspace=master%3A-; expires=Sat, 05 Aug 2023 18:40:17 GMT; path=/; secure; samesite=none"\n  </script>\n\n    \n<style id="critical" type="text/css">@charset "UTF-8";html{line-height:1.15;-ms-text-size-adjust:100%;-webkit-text-size-adjust:100%}body{margin:0}aside,nav,section{display:block}a{background-color:transparent;-webkit-text-decoration-skip:objects}img{border-style:none}svg:not(:root){overflow:hidden}button,input{font-family:sans-serif;font-size:100%;line-height:1.15;margin:0}button,input{overflow:visible}button{text-transform:none}[type=submit],button,html [type=button]{-webkit-appearance:button}[type=button]::-moz-focus-inner,[type=submit]::-moz-focus-inner,button::-moz-focus-inner{border-style:none;padding:0}[

In [191]:
# Parseamos la estructura con bs4
soup = BeautifulSoup(contenido_pagina, 'html.parser')
soup


<!DOCTYPE html>

<html lang="es-PE">
<head>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="vtex.render-server@8.168.0" name="generator"/>
<meta charset="utf-8"/>
<script>
    document.cookie = "VtexWorkspace=master%3A-; expires=Sat, 05 Aug 2023 18:40:17 GMT; path=/; secure; samesite=none"
  </script>
<style id="critical" type="text/css">@charset "UTF-8";html{line-height:1.15;-ms-text-size-adjust:100%;-webkit-text-size-adjust:100%}body{margin:0}aside,nav,section{display:block}a{background-color:transparent;-webkit-text-decoration-skip:objects}img{border-style:none}svg:not(:root){overflow:hidden}button,input{font-family:sans-serif;font-size:100%;line-height:1.15;margin:0}button,input{overflow:visible}button{text-transform:none}[type=submit],button,html [type=button]{-webkit-appearance:button}[type=button]::-moz-focus-inner,[type=submit]::-moz-focus-inner,button::-moz-focus-inner{border-style:none;padding:0}[type=button]:-moz-focusring,[type=submit]:

In [192]:
#Revisamos la estructura de la página y buscamos el componente que guarde la información de las tiendas
#Si analizamos la estructura y buscamos el nombre de las tiendas las encontramos en un script
# Este scrip esta dentro de <template  data-type="json" data-varname="__RUNTIME__"  data-field="extensions"  >
contenedor = soup.find('template', {'data-type': 'json', 'data-varname': '__RUNTIME__', 'data-field': 'extensions'})
contenedor

<template data-field="extensions" data-type="json" data-varname="__RUNTIME__">
<script>{"store.custom#nuestrastiendas":{"before":["$before_header.full"],"around":["$around_storeWrapper","$around_challenge"],"after":["$after_footer","$after_pixels"],"blockId":"vegaperu.store-theme@11.x:store.custom#nuestrastiendas","blocks":[{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#bannernuestrastiendas","extensionPointId":"flex-layout.row#bannernuestrastiendas","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#mapSearch","extensionPointId":"flex-layout.row#mapSearch","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#HorarioDeAtencion","extensionPointId":"flex-layout.row#HorarioDeAtencion","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#seccionsuscribete","extensionPointId":"flex-layout.row#seccionsuscribete","children":false,"blockRole":"block"}],"component":"vtex

In [193]:
# obtenemos el contenido del script
script_content = contenedor.script.string.strip()
script_content

'{"store.custom#nuestrastiendas":{"before":["$before_header.full"],"around":["$around_storeWrapper","$around_challenge"],"after":["$after_footer","$after_pixels"],"blockId":"vegaperu.store-theme@11.x:store.custom#nuestrastiendas","blocks":[{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#bannernuestrastiendas","extensionPointId":"flex-layout.row#bannernuestrastiendas","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#mapSearch","extensionPointId":"flex-layout.row#mapSearch","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#HorarioDeAtencion","extensionPointId":"flex-layout.row#HorarioDeAtencion","children":false,"blockRole":"block"},{"blockId":"vegaperu.store-theme@11.x:flex-layout.row#seccionsuscribete","extensionPointId":"flex-layout.row#seccionsuscribete","children":false,"blockRole":"block"}],"component":"vtex.render-runtime@8.134.0-beta.0\\u002FLayoutContainer","composition":"children","hasCon

In [194]:
# Lo convertimos a un diccionario de python
data_dict = json.loads(script_content)
data_dict

{'store.custom#nuestrastiendas': {'before': ['$before_header.full'],
  'around': ['$around_storeWrapper', '$around_challenge'],
  'after': ['$after_footer', '$after_pixels'],
  'blockId': 'vegaperu.store-theme@11.x:store.custom#nuestrastiendas',
  'blocks': [{'blockId': 'vegaperu.store-theme@11.x:flex-layout.row#bannernuestrastiendas',
    'extensionPointId': 'flex-layout.row#bannernuestrastiendas',
    'children': False,
    'blockRole': 'block'},
   {'blockId': 'vegaperu.store-theme@11.x:flex-layout.row#mapSearch',
    'extensionPointId': 'flex-layout.row#mapSearch',
    'children': False,
    'blockRole': 'block'},
   {'blockId': 'vegaperu.store-theme@11.x:flex-layout.row#HorarioDeAtencion',
    'extensionPointId': 'flex-layout.row#HorarioDeAtencion',
    'children': False,
    'blockRole': 'block'},
   {'blockId': 'vegaperu.store-theme@11.x:flex-layout.row#seccionsuscribete',
    'extensionPointId': 'flex-layout.row#seccionsuscribete',
    'children': False,
    'blockRole': 'block

In [195]:
# Necesitamos analizar el json y buscar dónde esta guardado los datos de las tiendas
lista_tiendas = data_dict["store.custom#nuestrastiendas/flex-layout.row#mapSearch/mapSearch"]["props"]["sucursales"]
lista_tiendas

[{'NOMBRE': 'Vega Mayorista Collique',
  'DIRECCION': 'Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. Collique',
  'TELEFONO': '981447496',
  'Correo': 'tiendamycollique@corporacionvega.pe',
  'Latitud': '-11.913586811296451',
  'Longitud': '-77.03157724269649'},
 {'NOMBRE': 'Vega Mayorista Infantas',
  'DIRECCION': 'Av. Gerardo Unger N° 6531 Urb. Infantas',
  'TELEFONO': '924493221',
  'Correo': 'tiendamyinfantas@corporacionvega.pe',
  'Latitud': '-11.945305283176683',
  'Longitud': '-77.06669732887826'},
 {'NOMBRE': 'Vega Mayorista Productores',
  'DIRECCION': 'Asociación de Productores Agricolas Mercado Santa Anita, Pj. B 13',
  'TELEFONO': '981173291',
  'Correo': 'tiendamyproductores@corporacionvega.pe',
  'Latitud': '-12.040767989379031',
  'Longitud': '-76.948008588956'},
 {'NOMBRE': 'Vega Supermayorista Belaunde',
  'DIRECCION': 'Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 cdra de la Av. Túpac Amaru)',
  'TELEFONO': '981445804',
  'Correo': 'tiendasmbelaunde@corporacionvega.pe

In [196]:
# Ahora mostramos la información del nombre y direccion
# necesitamos los campos de "NOMBRE " y "DIRECCION"
for tienda in lista_tiendas:
    print(f"Nombre de la tienda: {tienda['NOMBRE']} - Dirección: {tienda['DIRECCION']}")

Nombre de la tienda: Vega Mayorista Collique - Dirección: Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. Collique
Nombre de la tienda: Vega Mayorista Infantas - Dirección: Av. Gerardo Unger N° 6531 Urb. Infantas
Nombre de la tienda: Vega Mayorista Productores - Dirección: Asociación de Productores Agricolas Mercado Santa Anita, Pj. B 13
Nombre de la tienda: Vega Supermayorista Belaunde - Dirección: Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 cdra de la Av. Túpac Amaru)
Nombre de la tienda: Vega Supermayorista Santa Clara - Dirección: Av. Estrella 286 Urb. Santa Clara, (Centro Comercial Qhatu Plaza)
Nombre de la tienda: Vega Supermayorista San Antonio - Dirección: Fundo Punchauca Caudeguilla Mz. D, Lt. 1, (Alt. KM. 22 de la Av. Túpac Amaru)
Nombre de la tienda: Vega Supermayorista Chorrillos - Dirección: Jr. Genaro Numa Llona N °200, (Alt. de la cdras 4 y 5 de la Av Huaylas)
Nombre de la tienda: Vega Supermayorista Año Nuevo - Dirección: Jr. Jupiter Mz. 6, LT. 65-66, Urb. Villa Colliqu

In [197]:
list_direcciones=[d["DIRECCION"] for d in lista_tiendas]

In [198]:
list_nombres=[d["NOMBRE"] for d in lista_tiendas]
list_nombres

['Vega Mayorista Collique',
 'Vega Mayorista Infantas',
 'Vega Mayorista Productores',
 'Vega Supermayorista Belaunde',
 'Vega Supermayorista Santa Clara',
 'Vega Supermayorista San Antonio',
 'Vega Supermayorista Chorrillos',
 'Vega Supermayorista Año Nuevo',
 'Vega Supermayorista Colonial',
 'Vega Supermayorista Huamantanga',
 'Vega Supermayorista Ciudad y Campo',
 'Vega Supermayorista Sol De Naranjal',
 'Vega Supermayorista San Diego',
 'Vega Supermayorista Tomas Marsano',
 'Vega Supermayorista Lima VES',
 'Vega Supermayorista Minka',
 'Vega Market Aymaras',
 'Vega Market Olimpo',
 'Vega Market Bocanegra',
 'Vega Market Canta Callao',
 'Vega Market Mi Perú',
 'Vega Market Progreso',
 'Vega Market Santo Domingo',
 'Vega Market Tungasuca',
 'Vega Market Amaranto',
 'Vega Market Chorrillos Tupac',
 'Vega Market Malvinas',
 'Vega Market Año Nuevo',
 'Vega Market Mall Comas',
 'Vega Market Canevaro',
 'Vega Market Merino',
 'Vega Market Rospigliosi',
 'Vega Market Alayza',
 'Vega Market 

In [199]:
def get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService("chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [200]:
list_direcciones_final = []

driver = get_driver()

for  direccion in tqdm.tqdm(list_direcciones):

    dir = direccion + ", Peru"
    #dir = dir.replace('/', ' ').replace('#', ' ').replace('.', ' ')
    
    #driver.get("https://www.google.com/maps/search/"+dir)

    driver.get("https://www.google.com/maps/")

    searchbox = driver.find_element(By.ID, 'searchboxinput')

    searchbox.send_keys(dir)

    searchbutton = driver.find_element(By.ID, 'searchbox-searchbutton')

    searchbutton.click()

    try:
        time.sleep(3)
        coordenadas = driver.current_url
        

        latitud = float(coordenadas[coordenadas.find('@')+1:coordenadas.find(',', coordenadas.find('@')+2)])
        longitud =float(coordenadas[coordenadas.find('-', coordenadas.find('@')+2):coordenadas.find(',', coordenadas.find(',', coordenadas.find('@')+2)+1)])
        
    
        try:
            imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')
        
        except:
            print("Imagen no encontrada")

        nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

        try:
            direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text
            
        except:

            try:
                direccion_maps = driver.find_element(By.CSS_SELECTOR, "span.DkEaL").text
            
            except:
                direccion_maps = ''
                comentario = "Dirección no encontrada"
        
        comentario = 'Dirección encontrada'
    
    except:

        try:
            wait(driver, 4).until(ec.element_to_be_clickable((By.CLASS_NAME, "hfpxzc")))
            boton = driver.find_element(By.CLASS_NAME, "hfpxzc")
            boton.click()

            time.sleep(3)
            coordenadas = driver.current_url

            latitud = float(coordenadas[coordenadas.find('@')+1:coordenadas.find(',', coordenadas.find('@')+2)])
            longitud =float(coordenadas[coordenadas.find('-', coordenadas.find('@')+2):coordenadas.find(',', coordenadas.find(',', coordenadas.find('@')+2)+1)])

            try:
                imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')
            
            except:
                print("Imagen no encontrada")

            nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

            comentario = 'Más de una dirección encontrada, toma la primera'

            try:
                direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text
                
            except:

                try:
                    direccion_maps = driver.find_element(By.CSS_SELECTOR, "span.DkEaL").text
                
                except:
                    direccion_maps = ''
                    comentario = "Dirección no encontrada"
            
            print(comentario)

        except:
            
            comentario = "Dirección no encontrada"
            print(comentario)
        
            latitud = np.nan
            longitud = np.nan

            imagen = ''

            nombre_maps = ''

            direccion_maps = ''
            
            driver.get("https://www.google.com/maps")


    list_direcciones_final.append([direccion,imagen, latitud, longitud, comentario, nombre_maps, direccion_maps])

driver.close()

  0%|          | 0/58 [00:00<?, ?it/s]

Imagen no encontrada


  2%|▏         | 1/58 [00:10<10:22, 10.92s/it]

Más de una dirección encontrada, toma la primera


  3%|▎         | 2/58 [00:15<06:42,  7.18s/it]

...

 97%|█████████▋| 56/58 [06:44<00:12,  6.24s/it]

Imagen no encontrada
Dirección no encontrada


 98%|█████████▊| 57/58 [06:54<00:07,  7.30s/it]

Imagen no encontrada
Dirección no encontrada


100%|██████████| 58/58 [07:04<00:00,  7.31s/it]


In [204]:
list_direcciones_final

[['Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. Collique',
  'https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=VlddnGetlC9O1l2UldB09Q&cb_client=search.gws-prod.gps&w=408&h=240&yaw=92.45367&pitch=0&thumbfov=100',
  -11.9137167,
  -77.0364258,
  'Más de una dirección encontrada, toma la primera',
  'Lt. 01',
  'Av. Andres A Caceres 236, Comas 15324'],
 ['Av. Gerardo Unger N° 6531 Urb. Infantas',
  'https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=1Y64yaAdjmKPw57AYY-Q-A&cb_client=search.gws-prod.gps&w=408&h=240&yaw=265.6412&pitch=0&thumbfov=100',
  -11.9453415,
  -77.0693872,
  'Dirección encontrada',
  'Av. Gerardo Unger 6531',
  'Av. Gerardo Unger 6531, Comas 15314'],
 ['Asociación de Productores Agricolas Mercado Santa Anita, Pj. B 13',
  'https://lh5.googleusercontent.com/p/AF1QipNyFJZUaE53BJbmlfwPr-V5Tkcssj-KKzN-O57n=w533-h240-k-no',
  -12.0429276,
  -77.0411971,
  'Dirección no encontrada',
  'Santa Anita',
  ''],
 ['Av. Belaunde Oeste 198, Urb. Huaquill

In [205]:
df_tiendas_vega = pd.DataFrame(list_direcciones_final, columns=[ 'direccion', 'imagen', 'latitud', 'longitud', 'comentario', 'nombre_maps', 'direccion_maps'])
df_tiendas_vega

,direccion,imagen,latitud,longitud,comentario,nombre_maps,direccion_maps
0,"Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. ...",https://streetviewpixels-pa.googleapis.com/v1/...,-11.913717,-77.036426,"Más de una dirección encontrada, toma la primera",Lt. 01,"Av. Andres A Caceres 236, Comas 15324"
1,Av. Gerardo Unger N° 6531 Urb. Infantas,https://streetviewpixels-pa.googleapis.com/v1/...,-11.945341,-77.069387,Dirección encontrada,Av. Gerardo Unger 6531,"Av. Gerardo Unger 6531, Comas 15314"
2,Asociación de Productores Agricolas Mercado Sa...,https://lh5.googleusercontent.com/p/AF1QipNyFJ...,-12.042928,-77.041197,Dirección no encontrada,Santa Anita,
3,"Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 ...",https://streetviewpixels-pa.googleapis.com/v1/...,-11.939743,-77.056016,"Más de una dirección encontrada, toma la primera",Av. Victor Andrés Belaunde 198,"Av. Victor Andrés Belaunde 198, Comas 15312"
4,"Av. Estrella 286 Urb. Santa Clara, (Centro Com...",https://lh5.googleusercontent.com/p/AF1QipOxMN...,-12.018206,-76.886409,Dirección encontrada,Qhatu Plaza,"Av. La Estrella 286, Lima 15487"
5,"Fundo Punchauca Caudeguilla Mz. D, Lt. 1, (Alt...",https://lh5.googleusercontent.com/p/AF1QipOr3S...,-11.914876,-77.139375,"Más de una dirección encontrada, toma la primera",Casa Hacienda Punchauca,"Av. Túpac Amaru 5, Carabayllo 15320"
6,"Jr. Genaro Numa Llona N °200, (Alt. de la cdra...",https://streetviewpixels-pa.googleapis.com/v1/...,-12.167809,-77.028627,"Más de una dirección encontrada, toma la primera",Jirón Numa Llona 200,Confirma o corrige esta ubicación\nLa ubicació...
7,"Jr. Jupiter Mz. 6, LT. 65-66, Urb. Villa Colli...",https://lh5.googleusercontent.com/p/AF1QipP2Qp...,-11.916683,-77.094592,Dirección no encontrada,Collique,
8,"Av. Colonial 679, (Entre la Av. Colonial y Av....",,NaN,NaN,Dirección no encontrada,,
9,"Av. Puente Piedra 200, (MERCADO HUAMANTANGA)",https://lh5.googleusercontent.com/p/AF1QipNKaK...,-11.862631,-77.079087,"Más de una dirección encontrada, toma la primera",Vega Huamantanga,"Av. Puente Piedra 200, Puente Piedra 15118"


In [210]:
df_seleccion_Vega = df_tiendas_vega[['COMPETIDOR','direccion','latitud', 'longitud']]
df_seleccion_Vega

,COMPETIDOR,direccion,latitud,longitud
0,Vega,"Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. ...",-11.913717,-77.036426
1,Vega,Av. Gerardo Unger N° 6531 Urb. Infantas,-11.945341,-77.069387
2,Vega,Asociación de Productores Agricolas Mercado Sa...,-12.042928,-77.041197
3,Vega,"Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 ...",-11.939743,-77.056016
4,Vega,"Av. Estrella 286 Urb. Santa Clara, (Centro Com...",-12.018206,-76.886409
5,Vega,"Fundo Punchauca Caudeguilla Mz. D, Lt. 1, (Alt...",-11.914876,-77.139375
6,Vega,"Jr. Genaro Numa Llona N °200, (Alt. de la cdra...",-12.167809,-77.028627
7,Vega,"Jr. Jupiter Mz. 6, LT. 65-66, Urb. Villa Colli...",-11.916683,-77.094592
8,Vega,"Av. Colonial 679, (Entre la Av. Colonial y Av....",NaN,NaN
9,Vega,"Av. Puente Piedra 200, (MERCADO HUAMANTANGA)",-11.862631,-77.079087


In [211]:
df_tiendas_vega['COMPETIDOR']="Vega"

In [212]:
df_nombre = pd.DataFrame(list_nombres)

In [213]:
resultado = pd.concat([df_nombre,df_seleccion_Vega], ignore_index=True,axis=1)
re=resultado[[1, 0, 2, 3, 4]]
re


,1,0,2,3,4
0,Vega,Vega Mayorista Collique,"Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. ...",-11.913717,-77.036426
1,Vega,Vega Mayorista Infantas,Av. Gerardo Unger N° 6531 Urb. Infantas,-11.945341,-77.069387
2,Vega,Vega Mayorista Productores,Asociación de Productores Agricolas Mercado Sa...,-12.042928,-77.041197
3,Vega,Vega Supermayorista Belaunde,"Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 ...",-11.939743,-77.056016
4,Vega,Vega Supermayorista Santa Clara,"Av. Estrella 286 Urb. Santa Clara, (Centro Com...",-12.018206,-76.886409
5,Vega,Vega Supermayorista San Antonio,"Fundo Punchauca Caudeguilla Mz. D, Lt. 1, (Alt...",-11.914876,-77.139375
6,Vega,Vega Supermayorista Chorrillos,"Jr. Genaro Numa Llona N °200, (Alt. de la cdra...",-12.167809,-77.028627
7,Vega,Vega Supermayorista Año Nuevo,"Jr. Jupiter Mz. 6, LT. 65-66, Urb. Villa Colli...",-11.916683,-77.094592
8,Vega,Vega Supermayorista Colonial,"Av. Colonial 679, (Entre la Av. Colonial y Av....",NaN,NaN
9,Vega,Vega Supermayorista Huamantanga,"Av. Puente Piedra 200, (MERCADO HUAMANTANGA)",-11.862631,-77.079087


In [217]:
df_Vega = re.rename(columns={0:"NOMBRE_LOCAL",1:"COMPETIDOR",2:"DIRECCIÓN",3:"LATITUD",4:"LONGITUD"})
df_Vega

,COMPETIDOR,NOMBRE_LOCAL,DIRECCIÓN,LATITUD,LONGITUD
0,Vega,Vega Mayorista Collique,"Av. Andrés Avelino Cáceres N°236,Lt. 01. A.H. ...",-11.913717,-77.036426
1,Vega,Vega Mayorista Infantas,Av. Gerardo Unger N° 6531 Urb. Infantas,-11.945341,-77.069387
2,Vega,Vega Mayorista Productores,Asociación de Productores Agricolas Mercado Sa...,-12.042928,-77.041197
3,Vega,Vega Supermayorista Belaunde,"Av. Belaunde Oeste 198, Urb. Huaquillay, (A 1 ...",-11.939743,-77.056016
4,Vega,Vega Supermayorista Santa Clara,"Av. Estrella 286 Urb. Santa Clara, (Centro Com...",-12.018206,-76.886409
5,Vega,Vega Supermayorista San Antonio,"Fundo Punchauca Caudeguilla Mz. D, Lt. 1, (Alt...",-11.914876,-77.139375
6,Vega,Vega Supermayorista Chorrillos,"Jr. Genaro Numa Llona N °200, (Alt. de la cdra...",-12.167809,-77.028627
7,Vega,Vega Supermayorista Año Nuevo,"Jr. Jupiter Mz. 6, LT. 65-66, Urb. Villa Colli...",-11.916683,-77.094592
8,Vega,Vega Supermayorista Colonial,"Av. Colonial 679, (Entre la Av. Colonial y Av....",NaN,NaN
9,Vega,Vega Supermayorista Huamantanga,"Av. Puente Piedra 200, (MERCADO HUAMANTANGA)",-11.862631,-77.079087


In [218]:
df_Vega.to_csv('Vega.csv',index=False)